<a href="https://colab.research.google.com/github/DarshanaHeendeniya/ML_Learning/blob/main/Day1_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Authenticate in the Note book

import os
from google.colab import userdata
import google.generativeai as genai

try:
  GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
  os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY # For libraries that rely on env var
  genai.configure(api_key=GOOGLE_API_KEY)
  print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Colab secrets. Details: {e}")

✅ Gemini API key setup complete.


In [2]:
# Import the specific components from the Agent Development Kit and the Generative AI library.

from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
# Configure retry options

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

In [4]:
# Research Agent: Its job is to use the google_search tool and present findings.

research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [5]:
# Summarizer Agent: Its job is to summarize the text it receives.

summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


In [6]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.

root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


In [7]:
# Create the ochestrating environment (runner) and run the agent

runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI?"
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


ResearchCoordinator > The latest advancements in quantum computing are set to significantly impact AI. Quantum computers, with their ability to process information exponentially faster using qubits, can help AI overcome current limitations. This enhanced computational power will enable AI to solve complex problems that are currently intractable for classical computers, leading to breakthroughs in areas like drug discovery, materials science, and logistics optimization.

Furthermore, quantum computing promises to improve AI performance by making data processing more efficient, resulting in more accurate models. It also offers a more sustainable and energy-efficient path for training large AI models, addressing concerns about the high computational cost of current AI.

Specific AI fields like Natural Language Processing, drug discovery, healthcare, optimization, and autonomous vehicles are expected to be revolutionized. However, the field is still in its early stages, facing challenges i

## Sequential Workflows - The Assembly Line

Blog Post Creation with Sequential Agents



1.   Outline Agent - Creates a blog outline for a given topic
2.   Writer Agent - Writes a blog post


1.   Editor Agent - Edits a blog post draft for clarity and structure







In [8]:
# Outline Agent: Creates the initial blog post outline.

outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",  # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [9]:
# Writer Agent: Writes the full blog post based on the outline from the previous agent.

writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",  # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [10]:
# Editor Agent: Edits and polishes the draft from the writer agent.

editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",  # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


In [11]:
# Create root agent. Root agent is a sequential agent which runs it sub agents in the order they are listed.

root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [12]:
# Run the agent

runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post about the benefits of multi-agent systems for software developers"
)


 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers
OutlineAgent > ## Outline:

**Headline:** Supercharge Your Software: Unlocking the Power of Multi-Agent Systems

**Introduction Hook:** Tired of monolithic codebases that are hard to manage and scale? Imagine a world where your software components work together like a well-oiled machine, each with its own intelligence and purpose. That world is here, and it's called Multi-Agent Systems (MAS).

---

### Main Section 1: Enhanced Modularity and Specialization

*   **Decoupled Complexity:** Break down large, intricate problems into smaller, manageable agents, each responsible for a specific task or domain. This makes your codebase significantly easier to understand, develop, and debug.
*   **Focused Expertise:** Each agent can be designed and optimized for its particular function, leading to more efficient and specialized solutions. Think of them as micros

In [13]:
# Part of agent.py --> Follow https://google.github.io/adk-docs/get-started/quickstart/ to learn the setup

# --- 1. Define Sub-Agents for Each Pipeline Stage ---

# Code Writer Agent
# Takes the initial specification (from user query) and writes code.
code_writer_agent = Agent(
    name="CodeWriterAgent",
    model="gemini-2.5-flash-lite",
    # Change 3: Improved instruction
    instruction="""You are a Python Code Generator.
Based *only* on the user's request, write Python code that fulfills the requirement.
Output *only* the complete Python code block, enclosed in triple backticks (```python ... ```).
Do not add any other text before or after the code block.
""",
    description="Writes initial Python code based on a specification.",
    output_key="generated_code" # Stores output in state['generated_code']
)

# Code Reviewer Agent
# Takes the code generated by the previous agent (read from state) and provides feedback.
code_reviewer_agent = Agent(
    name="CodeReviewerAgent",
    model="gemini-2.5-flash-lite",
    # Change 3: Improved instruction, correctly using state key injection
    instruction="""You are an expert Python Code Reviewer.
    Your task is to provide constructive feedback on the provided code.

    **Code to Review:**
    ```python
    {generated_code}
    ```

**Review Criteria:**
1.  **Correctness:** Does the code work as intended? Are there logic errors?
2.  **Readability:** Is the code clear and easy to understand? Follows PEP 8 style guidelines?
3.  **Efficiency:** Is the code reasonably efficient? Any obvious performance bottlenecks?
4.  **Edge Cases:** Does the code handle potential edge cases or invalid inputs gracefully?
5.  **Best Practices:** Does the code follow common Python best practices?

**Output:**
Provide your feedback as a concise, bulleted list. Focus on the most important points for improvement.
If the code is excellent and requires no changes, simply state: "No major issues found."
Output *only* the review comments or the "No major issues" statement.
""",
    description="Reviews code and provides feedback.",
    output_key="review_comments", # Stores output in state['review_comments']
)


# Code Refactorer Agent
# Takes the original code and the review comments (read from state) and refactors the code.
code_refactorer_agent = Agent(
    name="CodeRefactorerAgent",
    model="gemini-2.5-flash-lite",
    # Change 3: Improved instruction, correctly using state key injection
    instruction="""You are a Python Code Refactoring AI.
Your goal is to improve the given Python code based on the provided review comments.

  **Original Code:**
  ```python
  {generated_code}
  ```

  **Review Comments:**
  {review_comments}

**Task:**
Carefully apply the suggestions from the review comments to refactor the original code.
If the review comments state "No major issues found," return the original code unchanged.
Ensure the final code is complete, functional, and includes necessary imports and docstrings.

**Output:**
Output *only* the final, refactored Python code block, enclosed in triple backticks (```python ... ```).
Do not add any other text before or after the code block.
""",
    description="Refactors code based on review comments.",
    output_key="refactored_code", # Stores output in state['refactored_code']
)


# --- 2. Create the SequentialAgent ---
# This agent orchestrates the pipeline by running the sub_agents in order.
code_pipeline_agent = SequentialAgent(
    name="CodePipelineAgent",
    sub_agents=[code_writer_agent, code_reviewer_agent, code_refactorer_agent],
    description="Executes a sequence of code writing, reviewing, and refactoring.",
    # The agents will run in the order provided: Writer -> Reviewer -> Refactorer
)

# For ADK tools compatibility, the root agent must be named `root_agent`
root_agent = code_pipeline_agent

# Run the agent

runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a python code to create a fitness app"
)


 ### Created new session: debug_session_id

User > Write a python code to create a fitness app
CodeWriterAgent > ```python
import datetime

class User:
    def __init__(self, name, age, gender, weight, height):
        self.name = name
        self.age = age
        self.gender = gender
        self.weight = weight  # in kg
        self.height = height  # in meters
        self.activity_log = []
        self.goals = {}

    def calculate_bmi(self):
        if self.height <= 0:
            return "Height must be positive."
        return self.weight / (self.height ** 2)

    def get_bmi_category(self):
        bmi = self.calculate_bmi()
        if isinstance(bmi, str):
            return bmi
        if bmi < 18.5:
            return "Underweight"
        elif 18.5 <= bmi < 24.9:
            return "Normal weight"
        elif 25 <= bmi < 29.9:
            return "Overweight"
        else:
            return "Obesity"

    def log_activity(self, activity_type, duration, calories_burned,

## Parallel Workflows - Independent Researchers



1.   Tech Researcher - Researches AI/ML news and trends
2.   Health Researcher - Researches recent medical news and trends


1.   Finance Researcher - Researches finance and fintech news and trends
2.   Aggregator Agent - Combines all research findings into a single summary







In [14]:
# Create tech_researcher agent: Focuses on AI and ML trends.

tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [15]:
# Creat health_researcher agent: Focuses on medical breakthroughs.

health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [16]:
# Create finance_researcher: Focuses on fintech trends.

finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [17]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.

aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}

    **Health Breakthroughs:**
    {health_research}

    **Finance Innovations:**
    {finance_research}

    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [18]:
# Create parellel and sequential agents.

# The ParallelAgent runs all its sub-agents simultaneously.

parallel_agent = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher]
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.

root_agent = SequentialAgent(
    name="ResearchPipeline",
    sub_agents=[parallel_agent, aggregator_agent]
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [19]:
# Create run time environment and run the agent

runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
TechResearcher > The latest AI/ML trends highlight advancements in **multimodal AI**, **agentic AI**, and **open-source AI**. Multimodal AI integrates various data types (text, audio, visual) for more nuanced understanding and interaction. Agentic AI focuses on proactive, autonomous agents capable of independent action, a shift from reactive systems. Open-source AI fosters global collaboration and transparency by allowing free use and modification of code.

Key companies heavily involved include **Google** (Gemini, Vertex AI, DeepMind), **OpenAI** (ChatGPT), **Microsoft** (Azure AI, Copilot), **Amazon** (AWS, Alexa), **Meta**, and **Apple**. NVIDIA is also crucial for GPU development powering these advancements.

The potential impact is transformative, enhancing human-computer interaction, automating complex tasks, and accelerating innovation across industries like healthca

## Loop Workflows - The Refinement Cycle

Iterative Story Refinement

1.   Writer Agent - Writes a draft of a short story
2.   Critic Agent - Reviews and critiques the short story to suggest improvements



In [26]:
# This agent runs ONCE at the beginning to create the first draft.

initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 10000-15000 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [21]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.

critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}

    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


In [22]:
# This is the function that the RefinerAgent will call to exit the loop.

def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}


print("✅ exit_loop function created.")

✅ exit_loop function created.


In [23]:
# This agent refines the story based on critique OR calls the exit_loop function.

refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a story refiner. You have a story draft and critique.

    Story Draft: {current_story}
    Critique: {critique}

    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


In [24]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.

story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.

root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [28]:
# Create a runner and run the root agent

runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"
)


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > Elias, his beard a frosted white against the salt-worn timbers of the lighthouse, polished the massive lens. For thirty years, the sea and sky had been his only companions, their moods more predictable than human folk. Tonight, however, was different. A storm raged, a fury of wind and wave that rattled his bones.

Amidst the chaos, a shard of emerald light pulsed from the seabed, visible even through the tempest’s spray. He scrambled down the winding stairs, his heart thrumming. There, lodged in the rocks, was not a piece of wreckage, but a smooth, obsidian tablet. As he touched it, intricate lines of pure, vibrant light bloomed across its surface, forming a map of stars he’d never seen.
CriticAgent > The plot sets up an intriguing mystery with the discovery of the obsidian tablet and its celestial map. The character of Elias is 